## Аналитика базы данных крупного сервиса для чтения книг (проект SQL)

### Описание проекта
Компания решила быть на волне изменений рынка из-за эпидемии короновируса и купила крупный сервис для чтения книг по подписке. Сейчас люди вынуждены отказаться от массовых развлечений и готовы потратить свое свободное время на чтение книг. Первая задача  аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Содержание

1. [Подготовка данных](#start)
2. [Анализ данных](#analysis)
3. [Вывод](#output)


<a id="start"></a>
### 1. Подготовка данных


In [2]:
import pandas as pd
from sqlalchemy import create_engine


Устанавливам параметры для подключения к базе данных

In [3]:
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db']) 

Сохраняем коннектор

In [4]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Последовательно откроем и изучим 5 таблиц из предоставленной базы данных в аналогичном порядке.

Таблица **books**

In [5]:
books = pd.io.sql.read_sql('''SELECT * FROM books''', con = engine) 

In [6]:
display(books.head(5))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Узнаем сколько строк в таблице

In [7]:
query_1 = '''SELECT count(*) 
             FROM books'''
pd.io.sql.read_sql(query_1, con = engine) 

,count
0,1000


Узнаем типы данных таблицы

In [8]:
query_2 = '''SELECT column_name, data_type 
             FROM information_schema.columns 
             WHERE table_name = 'books' '''
pd.io.sql.read_sql(query_2, con = engine) 

,column_name,data_type
0,book_id,integer
1,author_id,integer
2,title,text
3,num_pages,integer
4,publication_date,date
5,publisher_id,integer


Проверим, есть ли пропущенные значения.

In [9]:
query_3 = '''SELECT COUNT(*)
             FROM books
             WHERE book_id IS NULL
             OR author_id IS NULL
             OR title IS NULL
             OR num_pages IS NULL
             OR publication_date IS NULL
             OR publisher_id IS NULL'''
pd.io.sql.read_sql(query_3, con = engine) 

,count
0,0


В таблице books 6 столбцов и 1000 строк. Пропущенные значения отсутствуют. Типы данных корректны: целочисленный, строчный, дата.

Названия столбцов:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица **authors**

In [10]:
authors = pd.io.sql.read_sql('''SELECT * FROM authors LIMIT 5''', con = engine) 
display(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [11]:
query_4 = '''SELECT count(*) 
             FROM authors'''
pd.io.sql.read_sql(query_4, con = engine) 

,count
0,636


In [12]:
query_5 = '''SELECT column_name, data_type 
             FROM information_schema.columns 
             WHERE table_name = 'authors' '''
pd.io.sql.read_sql(query_5, con = engine) 

,column_name,data_type
0,author_id,integer
1,author,text


In [13]:
query_6 = '''SELECT COUNT(*)
             FROM authors
             WHERE author_id IS NULL
             OR author IS NULL'''
pd.io.sql.read_sql(query_6, con = engine) 

,count
0,0


В таблице authors 2 столбца и 636 строк. Пропущенные значения отсутствуют. Типы данных корректны: целочисленный, строчный.

Названия столбцов:
- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица **publishers**

In [14]:
publishers = pd.io.sql.read_sql('''SELECT * FROM publishers LIMIT 5''', con = engine) 
display(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [15]:
query_7 = '''SELECT count(*) 
             FROM publishers'''
pd.io.sql.read_sql(query_7, con = engine) 

,count
0,340


In [16]:
query_8 = '''SELECT column_name, data_type 
             FROM information_schema.columns 
             WHERE table_name = 'publishers' '''
pd.io.sql.read_sql(query_8, con = engine) 

,column_name,data_type
0,publisher_id,integer
1,publisher,text


In [17]:
query_9 = '''SELECT COUNT(*)
             FROM publishers
             WHERE publisher_id IS NULL
             OR publisher IS NULL'''
pd.io.sql.read_sql(query_9, con = engine) 

,count
0,0


В таблице publishers 2 столбца и 340 строк. Пропущенные значения отсутствуют. Типы данных корректны: целочисленный, строчный.

Названия столбцов:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства


Таблица **ratings**

In [18]:
ratings = pd.io.sql.read_sql('''SELECT * FROM ratings LIMIT 5''', con = engine) 
display(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [19]:
query_10 = '''SELECT count(*) 
              FROM ratings'''
pd.io.sql.read_sql(query_10, con = engine) 

,count
0,6456


In [20]:
query_11 = '''SELECT column_name, data_type 
              FROM information_schema.columns 
              WHERE table_name = 'ratings' '''
pd.io.sql.read_sql(query_11, con = engine) 

,column_name,data_type
0,rating_id,integer
1,book_id,integer
2,username,text
3,rating,integer


In [21]:
query_12 = '''SELECT COUNT(*)
              FROM ratings
              WHERE rating_id IS NULL
              OR book_id IS NULL
              OR username IS NULL
              OR rating IS NULL'''
pd.io.sql.read_sql(query_12, con = engine) 

,count
0,0


В таблице ratings 4 столбца и 6 456 строк. Пропущенные значения отсутствуют. Типы данных корректны: целочисленный, строчный.

Названия столбцов:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица **reviews**

In [22]:
reviews = pd.io.sql.read_sql('''SELECT * FROM reviews LIMIT 5''', con = engine) 
display(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [23]:
query_13 = '''SELECT count(*) 
              FROM reviews'''
pd.io.sql.read_sql(query_13, con = engine) 

,count
0,2793


In [24]:
query_14 = '''SELECT column_name, data_type 
              FROM information_schema.columns 
              WHERE table_name = 'reviews' '''
pd.io.sql.read_sql(query_14, con = engine) 

,column_name,data_type
0,review_id,integer
1,book_id,integer
2,username,text
3,text,text


In [25]:
query_15 = '''SELECT COUNT(*)
              FROM reviews
              WHERE review_id IS NULL
              OR book_id IS NULL
              OR username IS NULL
              OR text IS NULL'''
pd.io.sql.read_sql(query_15, con = engine) 

,count
0,0


В таблице reviews 4 столбца и 2 793 строки. Пропущенные значения отсутствуют. Типы данных корректны: целочисленный, строчный.

Названия столбцов:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

Данные в таблицах достаточно "чистые". Нет пропусков. Типы данных корректны.

<a id="analysis"></a>
### 2. Анализ данных


**Посчитаем, сколько книг вышло после 1 января 2000 года.**

In [26]:
query_16 = '''SELECT COUNT(book_id) AS book_quantity
              FROM books
              WHERE publication_date > '2000-01-01' '''
pd.io.sql.read_sql(query_16, con = engine)


,book_quantity
0,819


Согласно нашим данным всего в базе _1000 книг, из них 819_ выпущено после 1 января 2000 года, т.е. 82% всех книг в базе данных - это современные издания, написанные или переизданные с учетом корректировок уже в XXI веке. Это важный фактор для привлечения пользователей. Возможно, в базе будет много новинок или книг, ранее неизвестных русскоязычному читателю.

**Для каждой книги посчитаем количество обзоров и среднюю оценку.**

In [27]:
query_17 = '''SELECT 
                  DISTINCT books.book_id AS book_id,
                  books.title AS book_title,
                  authors.author AS author, 
                  AVG(ratings.rating) AS average_rating,
                  COUNT(DISTINCT reviews.review_id) AS reviews_quantity
              FROM 
                  books
              INNER JOIN ratings ON ratings.book_id = books.book_id
              INNER JOIN authors ON authors.author_id = books.author_id
              INNER JOIN reviews ON reviews.book_id = books.book_id 
              GROUP BY 
                  books.book_id,
                  authors.author
              ORDER BY 
                  reviews_quantity DESC
           '''
pd.io.sql.read_sql(query_17, con = engine)

,book_id,book_title,author,average_rating,reviews_quantity
0,948,Twilight (Twilight #1),Stephenie Meyer,3.662500,7
1,207,Eat Pray Love,Elizabeth Gilbert,3.395833,6
2,299,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling/Mary GrandPré,4.287500,6
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.414634,6
4,497,Outlander (Outlander #1),Diana Gabaldon,4.125000,6
...,...,...,...,...,...
989,872,The Social Contract,Jean-Jacques Rousseau/Maurice Cranston,3.500000,1
990,930,To Green Angel Tower (Memory Sorrow and Thor...,Tad Williams,4.500000,1
991,943,Tsubasa: RESERVoir CHRoNiCLE Vol. 1,CLAMP/Anthony Gerard,2.500000,1
992,980,Wicked: The Grimmerie,David Cote/Stephen Schwartz/Joan Marcus/Winnie...,3.500000,1


По количеству обзоров безусловным лидером является книга *С.Майер "Сумерки"* -7. В ТОП-5 также входят книги Э.Гилберт "Ешь, молись, люби", книги Дж.Роулинг о Гарри Поттере - по 6 отзывов. В конце списка книги, имеющие по одному отзыву. В целом, есть около 6 книг без отзывов - это всего 0,6% от количества книг.

Посмотрим рейтинг книг по оценкам пользователей.

In [28]:
query_18 = '''SELECT 
                  books.title AS title,
                  authors.author AS author, 
                  AVG(ratings.rating) AS average_rating,
                  COUNT(DISTINCT reviews.review_id) AS reviews_quantity
               FROM 
                  books
              INNER JOIN ratings ON ratings.book_id = books.book_id
              INNER JOIN authors ON authors.author_id = books.author_id
              LEFT JOIN reviews ON reviews.book_id = books.book_id 
              GROUP BY 
                  books.title,
                  authors.author
              ORDER BY 
                  average_rating DESC
           '''
pd.io.sql.read_sql(query_18, con = engine)

,title,author,average_rating,reviews_quantity
0,Pop Goes the Weasel (Alex Cross #5),James Patterson,5.00,2
1,Angels Fall,Nora Roberts,5.00,2
2,Piercing the Darkness (Darkness #2),Frank E. Peretti,5.00,2
3,The Cat in the Hat and Other Dr. Seuss Favorites,Dr. Seuss/Various,5.00,0
4,Neil Gaiman's Neverwhere,Mike Carey/Glenn Fabry/Neil Gaiman,5.00,2
...,...,...,...,...
994,The World Is Flat: A Brief History of the Twen...,Thomas L. Friedman,2.25,3
995,Junky,William S. Burroughs/Oliver Harris/Allen Gins...,2.00,2
996,Drowning Ruth,Christina Schwarz,2.00,3
997,His Excellency: George Washington,Joseph J. Ellis,2.00,2


Самая высокая оценка пользователя - 5, самая низкая - 1.5. Все книги имеют рейтинг. В ТОП-5 входят: К.Мур "Грязная работа", К.Харрисон "За пригоршню чар", Б.Тейлор "Женщина субстанции", В. Флинн "Акт измены", П.Фрэнк "Вавилон". На последнем месте рейтинга Д.Пиколт "Урожай сердца".

**Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — с целью исключить из анализа брошюры**

In [29]:

query_19 = '''SELECT 
                 DISTINCT publishers.publisher AS publisher,
                 COUNT (books.book_id) AS book_quantity
              FROM 
                 books
              INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
              WHERE
                 num_pages > 50
              GROUP BY
                 publisher
              ORDER BY
                 book_quantity DESC'''
                         
pd.io.sql.read_sql(query_19, con = engine)

,publisher,book_quantity
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Wordsworth Editions,1
330,Workman Publishing Company,1
331,Wyatt Book,1
332,Yale University Press,1


Из 340 издательств в базе данных 334 выпускают книги с более чем 50 страницами. Самое результативное из них это издательство - *Penguin Books* (42 книги), британское издательство, выпускающее популярную в мягких обложках. Далее, идут *Vintage* (31 книга), также основанное в Англии и выпускающее современную беллетристику и американское издательство *Grand Central Publishing* (25 книг). Менее всего (по 1 книге) у издательств	Wyatt Book, Yale University Press, Yearling Books.	

**Определим автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками**

In [30]:
query_20 = ''' SELECT 
                  Sub.author_1 AS author,
                  AVG(Sub.average_rating_1) AS average_rating
               FROM
                  (SELECT 
                      DISTINCT books.title AS books, 
                      COUNT(ratings.rating) AS rating_quantity,
                      authors.author AS author_1,
                      AVG(ratings.rating) AS average_rating_1
                   FROM 
                      books
                   INNER JOIN ratings ON ratings.book_id = books.book_id
                   INNER JOIN authors ON authors.author_id = books.author_id
                   GROUP BY 
                      books,
                      author_1
                   HAVING COUNT(ratings.rating) >= 50) AS Sub
                   GROUP BY
                      author
                   ORDER BY
                      average_rating DESC'''

pd.io.sql.read_sql(query_20, con = engine)


,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Arthur Golden,4.107143
5,Rick Riordan,4.080645
6,William Golding,3.901408
7,J.D. Salinger,3.825581
8,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
9,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879


Всего 14 авторов, книги которых получили 50 и более оценок. Самая высокая средняя оценка у книг Дж.Роулинг (4.28). В ТОП- 5 входят с рейтингом 4.1-4.26: _Маркус Зусак, Дж.Р.Толкин, Л.Олкотт, А.Голден_. Самый низкий рейтинг у Дж.Стейнбека (3.62) и С.Майер (3.66), хотя "Сумерки" на втором месте по числу обзоров.

**Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

In [71]:
query_21 = '''SELECT
                  AVG(Sub.review_quantity) AS average_reviews
              FROM
                  (SELECT
                       COUNT(DISTINCT reviews.review_id) AS review_quantity 
                   FROM  
                       reviews
                   WHERE username IN
                       (SELECT 
                              DISTINCT ratings.username AS username
                        FROM 
                              ratings
                        GROUP BY 
                              ratings.username
                        HAVING
                             COUNT(ratings.rating) > 50) 
                    GROUP BY
                        username) AS Sub
            '''

pd.io.sql.read_sql(query_21, con = engine)

,average_reviews
0,24.333333


Количество пользователей, которые поставили больше 50 оценок, составляет 6.

Данные пользователи написали 146 обзоров.

Среднее количество обзоров от данных пользователей составляет 24.3.

In [39]:
#КОД РЕВЬЮЕРА

query_21 = '''SELECT 
                              DISTINCT reviews.username AS username, 
                              COUNT(ratings.rating) as cnt_ratings,
                              COUNT(reviews.review_id) as cnt_review
                        FROM 
                              reviews
                        INNER JOIN ratings ON ratings.book_id = reviews.book_id
                        GROUP BY 
                              reviews.username
                        HAVING
                             COUNT(ratings.rating) > 50
          '''

pd.io.sql.read_sql(query_21, con = engine)

,username,cnt_ratings,cnt_review
0,abbottjames,148,148
1,abigailmoore,195,195
2,adamsabigail,255,255
3,alison92,74,74
4,amy97,215,215
...,...,...,...
151,yvonnevillarreal,163,163
152,yweeks,89,89
153,zjohnston,199,199
154,znelson,68,68


<a id="output"></a>
### 3. Вывод



Проанализировав информацию из предоставленной базы данных о книгах, издательствах, авторах, пользовательских обзорах и оценках, можно заключить следующее:

1. **Количество книг и период их издания:** всего в базе данных _1000 книг, из них 819_ выпущено после 1 января 2000 года, т.е. 82% всех книг в базе данных - это современные издания, написанные или переизданные с учетом корректировок уже в XXI веке. Это важный фактор для привлечения пользователей. 
Возможно, в базе будет много новинок или книг, ранее неизвестных русскоязычному читателю.

2. **Лучшие издательства:** из 340 издательств в базе данных 334 выпускают книги с более чем 50 страницами. Самое результативное из них это издательство - *Penguin Books* (42 книги), британское издательство, выпускающее популярную в мягких обложках. Далее, идут *Vintage* (31 книга), также основанное в Англии и выпускающее современную беллетристику и американское издательство *Grand Central Publishing* (25 книг). 
Менее всего (по 1 книге) у издательств	Wyatt Book, Yale University Press, Yearling Books.	

3. **Количество читательских обзоров:** одним из плюсов сервиса является читательский опыт и рекомендации по большинству книг. По количеству обзоров безусловным лидером является книга *С.Майер "Сумерки"* -7. В ТОП-5 также входят книги Э.Гилберт "Ешь, молись, люби", книги Дж.Роулинг о Гарри Поттере - по 6 отзывов. В конце списка книги, имеющие по одному отзыву. В целом, есть около 6 книг без отзывов - это всего 0,6% от количества книг.

4. **Оценки читателей:** сервис предоставляет возможность знакомиться с оценками как самих книг, так и их авторов.
  
  _Самая высокая оценка книги_ - 5 баллов, самая низкая - 1.5. Все книги имеют рейтинг. В ТОП-5 входят: К.Мур "Грязная работа", К.Харрисон "За пригоршню чар", Б.Тейлор "Женщина субстанции", В. Флинн "Акт измены", П.Фрэнк "Вавилон". На последнем месте рейтинга Д.Пиколт "Урожай сердца".
  
  _Всего 14 авторов,_ книги которых получили 50 и более оценок. Самая высокая средняя оценка у книг Дж.Роулинг (4.28). В ТОП- 5 входят с рейтингом 4.1-4.26: _Маркус Зусак, Дж.Р.Толкин, Л.Олкотт, А.Голден_. Самый низкий рейтинг у Дж.Стейнбека (3.62) и С.Майер (3.66), хотя "Сумерки" на втором месте по числу обзоров.

5. **Активность пользователей:** Самыме активные пользователи, которые прочитали и оценили более 50 книг, составили в среднем 24,3 обзоров. 

Таким образом, наше **ценностное предложение** состоит из следующих пунктов:
- Как данный сервис решает проблемы клиентов?

Удобство, доступность 24/7, возможность нетворкинга с другими пользователями-книголюбами.

- В чем его преимущества?

Современная библиотека, новинки, популярная литература на английском языке, система читательских оценок и обзоров, возможность быстрого выбора после ознакомления с читательским опытом других пользователей.

- Чем вы отличаетесь от конкурентов?

82% современной литературы с годом издании не позднее 2000 г., книги от лучших европейских и американских издательств, являющихся лидерами рынками на протяжении многих десятилетий, быстрый доступ к мировым бестселлерам. 